In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_helper import get_list_of_index_names, broadcast_simple, concat_categories

%matplotlib inline

This is just for SIDS at the moment. Can be updated for all countries when we have the data

In [67]:
#Names to WB names
any_to_wb = pd.read_csv("inputs/any_name_to_wb_name.csv",index_col="any",squeeze=True)

#GAR names with SIDS spec
gar_name_sids = pd.read_csv("inputs/gar_name_sids.csv")

#iso3 to wb country name table
iso3_to_wb=pd.read_csv("inputs/iso3_to_wb_name.csv").set_index("iso3").squeeze()

#iso2 to iso3 table
iso2_iso3 = pd.read_csv("inputs/names_to_iso.csv", usecols=["iso2","iso3"]).drop_duplicates().set_index("iso2").squeeze() #the tables has more lines than countries to account for several ways of writing country names

In [68]:
path = "sids data\\"

## Roads -- km from OSM and hard coding road costs

In [69]:
roads = pd.read_csv(path+"km_roads_sids.csv")

In [70]:
roads[['Total Length (km)', 'Total Primary',\
       'Total Secondary', 'Total Tertiary', 'Residential', 'Track / service',\
       'Total Other']] = roads[['Total Length (km)', 'Total Primary',\
       'Total Secondary', 'Total Tertiary', 'Residential', 'Track / service',\
       'Total Other']].astype(float)

In [71]:
roads["country"] = roads.country.replace(any_to_wb)

In [72]:
#cost per km of road based on st Lucia data. 
primary_road_cost = 1200000
secondary_road_cost = 700000
tertiary_road_cost = 600000

In [73]:
#reduced road values because of depreciation.
lifetime_primary = 20
lifetime_secondary = 15
lifetime_tertiary = 10

In [74]:
roads = roads.fillna(0)
roads = roads.set_index("country")

In [75]:
roads["average_age_primary"] = 8
roads["average_age_secondary"] = 5
roads["average_age_tertiary"] = 5

In [76]:
roads.loc["Kiribati","average_age_primary"]=12

In [77]:
roads["transport"] = roads["Total Primary"]*primary_road_cost*(1-1/lifetime_primary*roads.average_age_primary)+\
                             roads["Total Secondary"]*secondary_road_cost*(1-1/lifetime_secondary*roads.average_age_secondary)+\
                             roads["Total Tertiary"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Residential"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Track / service"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Total Other"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)

## Power (from Platts)

In [78]:
power = pd.read_csv(path+"power_plants_sids.csv")
power["country"] = power.COUNTRY.apply(lambda x:x.title()).replace(any_to_wb)

In [79]:
power["power"] = power.CAPEX

In [80]:
power.set_index("country").power

country
American Samoa                    3.269578e+07
Anguilla                          1.252212e+07
Antigua and Barbuda               4.313640e+07
Aruba                             9.078911e+07
Bahamas, The                      3.677894e+08
Bahrain                           1.121848e+10
Barbados                          1.298503e+08
Belize                            2.467837e+08
Bermuda                           7.038958e+07
Virgin Islands, British           2.624483e+07
Cape Verde                        1.129964e+08
Comoros                           4.821260e+06
Cook Islands                      8.042685e+06
Dominica                          2.400291e+07
Dominican Republic                3.796155e+09
Fiji                              4.830101e+08
Guam                              2.402088e+08
Guinea-Bissau                     1.467094e+07
Guyana                            1.555213e+08
Haiti                             2.236963e+08
Jamaica                           7.874247e+08
Kirib

## Buildings data from Rashmin and PCRAFI 

In [81]:
buildings_pcrafi = pd.read_csv(path+"PCRAFI_stocks.csv")
#values in million USD

In [82]:
#we divide by 3 because for Fiji PCRAFI is 3 times higher than Rashmin's assessment, which is consistent with GAR
buildings_pcrafi["country"] = buildings_pcrafi.country.replace(any_to_wb)
buildings_pcrafi = 1/3*10**6*buildings_pcrafi.set_index("country")

In [83]:
#buildings_pcrafi.rename(columns={"building":"total_building_value"},inplace=True)

In [84]:
buildings_pcrafi

,building,infrastructure
country,,
Cook Islands,4.322667e+08,3.920000e+07
Fiji,6.288400e+09,1.031300e+09
"Micronesia, Fed. Sts.",5.763333e+08,1.042667e+08
Kiribati,3.353667e+08,5.473333e+07
Marshall Islands,4.680333e+08,9.530000e+07
Nauru,5.793333e+07,2.466667e+07
Niue,1.368667e+08,1.400000e+07
Papua New Guinea,1.316967e+10,2.213033e+09
Palau,4.461667e+08,5.330000e+07


In [85]:
buildings_rashmin = pd.read_csv(path+"Gross_Capital_Summary.csv")
buildings_rashmin["country"] = buildings_rashmin.country.replace(iso3_to_wb).replace(any_to_wb)

In [86]:
buildings_rashmin

,country,Private K,Public K,Buildings (Residential + Non-Res),Infrastructure
0,Belize,0.686881,0.313119,4.620789e+09,NaN
1,St. Lucia,0.785803,0.214197,3.197446e+09,1.169627e+09
2,Jamaica,0.788723,0.211277,3.822092e+10,1.151195e+10
3,Haiti,0.733435,0.266565,2.656431e+10,8.862872e+09
4,Grenada,0.794724,0.205276,2.131954e+09,NaN
5,Dominican Republic,0.759619,0.240381,1.530000e+11,NaN
6,Fiji,0.504894,0.495106,6.504790e+09,NaN
7,FJI est 2,0.593038,0.406962,NaN,NaN


In [87]:
buildings_rashmin["building"] = buildings_rashmin["Buildings (Residential + Non-Res)"]

In [88]:
buildings_rashmin.set_index("country").building

country
Belize                4.620789e+09
St. Lucia             3.197446e+09
Jamaica               3.822092e+10
Haiti                 2.656431e+10
Grenada               2.131954e+09
Dominican Republic    1.530000e+11
Fiji                  6.504790e+09
FJI est 2                      NaN
Name: building, dtype: float64

In [89]:
#Fiji has 2 estimates. first one is rashmin, last one is PCRAFI
buildings = pd.concat([buildings_rashmin.set_index("country"),buildings_pcrafi]).reset_index().drop_duplicates("country",keep="first")

### Residential vs non residential buildings

In [90]:
buildings = buildings.set_index("country")
## hard coding Fiji data from Rashmin's file Fiji_2015_exposure_estimation_110517.xlsx
fiji_total = 4093521135+2411268610
fiji_residential = 4093521135/fiji_total

# for the US, according to the BEA, residential is 19,199.1 billion and non residential 21,879.4 billion. so by default we put 50/50.
buildings["building_residential"] = buildings.building*0.5
buildings["building_non_residential"] = buildings.building*0.5
buildings.loc["Fiji","building_residential"] = buildings.loc["Fiji","building"]*fiji_residential
buildings.loc["Fiji","building_non_residential"] = buildings.loc["Fiji","building"]*(1-fiji_residential)

## Water from WASH infrastructure needs

In [91]:
water = pd.read_csv(path+"water_infra_value_sids.csv")

In [92]:
water["country"] = water["country"].replace(any_to_wb)
water.set_index("country").sample()

,water
country,
St. Vincent and the Grenadines,101050957.6


## Merge all available infra 

In [93]:
all_infra = pd.concat([roads.transport,power.set_index("country").power,\
                       buildings[["building_residential","building_non_residential"]],water.set_index("country")],axis=1)

In [94]:
all_infra.sample(3)

,transport,power,building_residential,building_non_residential,water
Comoros,NaN,4821260.0,NaN,NaN,2.001271e+08
Marshall Islands,3.323244e+07,15285248.0,2.340167e+08,2.340167e+08,7.076137e+06
Singapore,NaN,NaN,NaN,NaN,NaN


In [95]:
all_infra["infra"] = all_infra.power+all_infra.water+all_infra.transport
all_infra["infra_and_buildings"] = all_infra.power+all_infra.water+all_infra.transport+all_infra.building_non_residential+all_infra.building_residential
all_infra['buildings'] = all_infra.building_non_residential+all_infra.building_residential

In [96]:
all_infra.dropna()

,transport,power,building_residential,building_non_residential,water,infra,infra_and_buildings,buildings
Belize,2.513105e+09,246783703.0,2.310395e+09,2.310395e+09,4.352581e+08,3.195147e+09,7.815936e+09,4.620789e+09
Fiji,2.981647e+09,483010130.0,4.093521e+09,2.411269e+09,4.135154e+08,3.878172e+09,1.038296e+10,6.504790e+09
Haiti,9.742686e+09,223696330.0,1.328215e+10,1.328215e+10,1.271568e+09,1.123795e+10,3.780226e+10,2.656431e+10
Jamaica,5.119131e+09,787424726.0,1.911046e+10,1.911046e+10,4.261407e+09,1.016796e+10,4.838888e+10,3.822092e+10
Kiribati,3.843400e+08,4982963.0,1.676833e+08,1.676833e+08,1.082715e+07,4.001501e+08,7.355168e+08,3.353667e+08
Marshall Islands,3.323244e+07,15285248.0,2.340167e+08,2.340167e+08,7.076137e+06,5.559382e+07,5.236272e+08,4.680333e+08
"Micronesia, Fed. Sts.",1.575350e+08,20352430.0,2.881667e+08,2.881667e+08,1.600597e+07,1.938934e+08,7.702267e+08,5.763333e+08
Papua New Guinea,6.976123e+09,889191070.0,6.584833e+09,6.584833e+09,8.536102e+08,8.718925e+09,2.188859e+10,1.316967e+10
Samoa,9.025303e+08,47664858.0,3.579833e+08,3.579833e+08,7.177621e+07,1.021971e+09,1.737938e+09,7.159667e+08
Solomon Islands,5.905869e+08,27550125.0,5.097833e+08,5.097833e+08,5.018003e+07,6.683170e+08,1.687884e+09,1.019567e+09


## Read total capital data gathered in "gather_capital_data" and wb data gathered in "download_wb_data"

In [97]:
K = pd.read_csv("intermediate/avg_prod_k_with_gar_for_sids.csv",index_col="Unnamed: 0")
wb = pd.read_csv("inputs/wb_data.csv",index_col="country")

In [98]:
names_to_iso2 = pd.read_csv("inputs/names_to_iso.csv", usecols=["iso2","country"]).drop_duplicates().set_index("country").squeeze()
wb["Ktot"] = wb.gdp_pc_cd*wb['pop']/K.avg_prod_k
wb["GDP"] = wb.gdp_pc_cd*wb['pop']
wb["avg_prod_k"] = K.avg_prod_k
wb['iso2'] = names_to_iso2


## recomputes Ktot from WB GDP and average capital productivity

In [99]:
wb["Ktot"] = wb.gdp_pc_cd*wb['pop']/K.avg_prod_k
wb["GDP"] = wb.gdp_pc_cd*wb['pop']

In [100]:
all_infra[['Ktot','GDP']] = wb[['Ktot','GDP']]
aa = all_infra.dropna()

In [101]:
aa.infra_and_buildings/aa.Ktot

Belize              1.185212
Fiji                0.780363
Haiti               0.628418
Jamaica             0.700667
Kiribati            1.259914
Papua New Guinea    0.403335
Solomon Islands     0.416853
St. Lucia           1.148905
Tonga               1.325876
dtype: float64

In [102]:
shares_K = all_infra.multiply(1/all_infra.Ktot,axis="index")
shares_GDP = all_infra.multiply(1/all_infra.GDP,axis="index")

In [104]:
shares_GDP[["transport","power","water","buildings"]].to_csv("infra_share_GDP_MER.csv")

In [105]:
pp = all_infra[['transport','Ktot','GDP']].dropna()

In [106]:
pp['transport_pc_K'] = pp.transport/pp.Ktot
pp['transport_pc_gdp'] = pp.transport/pp.GDP

In [107]:
pp

,transport,Ktot,GDP,transport_pc_K,transport_pc_gdp
Antigua and Barbuda,4.662181e+08,7.371866e+09,1.449160e+09,0.063243,0.321716
"Bahamas, The",2.480946e+09,5.078488e+10,9.047000e+09,0.048852,0.274229
Barbados,9.070417e+08,1.524207e+10,4.587550e+09,0.059509,0.197718
Belize,2.513105e+09,6.594545e+09,1.765256e+09,0.381088,1.423649
Bermuda,2.367476e+08,3.009187e+10,5.602007e+09,0.007867,0.042261
Cuba,2.442253e+10,2.236503e+11,8.724341e+10,0.109200,0.279935
Dominica,4.304822e+08,2.110774e+09,5.254246e+08,0.203945,0.819304
Dominican Republic,1.065118e+10,2.387625e+11,7.158355e+10,0.044610,0.148794
Fiji,2.981647e+09,1.330530e+10,4.631626e+09,0.224095,0.643758
Haiti,9.742686e+09,6.015462e+10,8.022639e+09,0.161961,1.214399
